# Counts the number of simulations with each parameter set

In [1]:

import os
from collections import OrderedDict
from random import sample
from tqdm import tqdm_notebook
import logging
import pandas as pd
from math import floor
import shutil
import pathlib
import sqlite3


In [10]:
# For dev use only - auto-reloading of modules
%load_ext autoreload
%aimport pycoalescence.coalescence_tree
import pathlib

from pycoalescence.coalescence_tree import check_sql_table_exist
from pycoalescence.helper import update_parameter_names
from pycoalescence import CoalescenceTree
from pycoalescence.sqlite_connection import check_sql_table_exist, SQLiteConnection
from compress_times import compress_times, double_backup, double_restore_backup, get_unique_times

%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

intervals = {'artinskian',
 'asselian',
 'bashkirian',
 'gzhelian',
 'kasimovian',
 'kungurian',
 'moscovian',
 'sakmarian'}

tetrapod_groups = {'amniote', 'amphibian'}

In [4]:
def percent_cover_detection(fine_map_file):
	"""
	Detects the percentage cover from the fine map path
	:param fine_map_file: path to check for
	:return: the simulation percentage cover
	"""
	for pc in [0.1, 0.2, 0.4, 0.8]:
		if "_{}_".format(pc) in fine_map_file:
			return pc
	raise ValueError("No percentage cover detected!")

In [5]:
def sim_type_detection(fine_map_file):
	"""
	Detects the simulation type from the fine map path (because I didn't save it anywhere sensible!
	:param fine_map_file: the path to check for
	:return the sim type
	"""
	for interval in intervals:
		for tet_group in tetrapod_groups:
			if interval in fine_map_file and tet_group in fine_map_file:
				return (interval, tet_group)
	raise ValueError("No type detected! Filename: {}.".format(fine_map_file))

In [6]:
def sim_scenario_detection(file):
    """
    Detects the scenario from the file name
    :param the file name to check
    :return the scenario
    """
    for scenario in ["fragmented", "clustered", "pristine"]:
        if scenario in file:
            return scenario
    raise ValueError("Scenario not detected")

In [7]:

# Set the import directories and variables - paths relative to the jupyter notebook directory
# fragmented="fragmented"
fragmented = "all"  # put back if want to analyse normal sims
local_dir = "/home/sam/Documents/PhD/PaleoSampling"
ext_dir = "/run/media/sam/Media/Paleo"
# local_dir = "/Users/samthompson/Documents/PhD/PaleoSampling/"
# ext_dir = "/Volumes/Seagate 3TB/Paleo/"
results_dir = os.path.join(ext_dir, "Results", "PaleoMainOcc2b")
dst_folder = os.path.join(local_dir, "Results", "PaleoMainOcc2")
dst_folder2 = os.path.join(
    local_dir, "Code", "MainSimulationR", "results", "PaleoMainOcc2"
)
data_dir = os.path.join(ext_dir, "Data")
speciation_rates = [
    0.000_000_01,
    0.000_000_001,
    0.000_000_002,
    0.000_000_004,
    0.000_000_006,
    0.000_000_008,
    0.000_000_05,
    0.000_000_1,
    0.000_001,
]
sim_int = 2
directory_descriptions = [
    ("Main{}".format(sim_int), "pristine"),
    ("Fragmented{}".format(sim_int), "fragmented"),
    ("Clustered{}".format(sim_int), "clustered"),
]


In [12]:
parameters = []
for file in pathlib.Path(results_dir).iterdir():
    if file.suffix == ".db":
        try:
            ct = CoalescenceTree(file)
        except:
            continue
        params = ct.get_simulation_parameters()
        parameters.append({""})